In [6]:
import os
import cv2
import shutil
import urllib.request
from PIL import Image, ImageFilter

In [7]:
PATH_TO_STORED_CARTOON_IMAGES = './drive/MyDrive/Project/safebooru/'
PATH_TO_STORE_SMOOTHED_IMAGES = './drive/MyDrive/Project/safebooru_smoothed/'
SMOOTHED_IMAGES_ZIPFILE_NAME = './drive/MyDrive/Project/safebooru_smoothed'

In [8]:
def init_counter(path_to_count_content):
    if not os.path.exists(path_to_count_content):
        return 0
    else:
        already_existing_no_of_files = len(os.listdir(path_to_count_content))
        print("Already downloaded files: ", already_existing_no_of_files)
        return already_existing_no_of_files

def download_non_already_existing_images(filename_with_path, url):
    if not os.path.exists(filename_with_path):
        return download_image(url, filename_with_path)
    else:
        print("Skipping file, already exists: ", filename_with_path)
        return False

def download_image(url, filename):
    try:
        with urllib.request.urlopen(url) as response, open(filename, 'wb') as saving_file:
            shutil.copyfileobj(response, saving_file)
            return True
    except urllib.error.HTTPError as e:
        print(e)
        return False

def zip_images(zipfile_name, path_to_data_to_be_zipped):
    shutil.make_archive(zipfile_name, 'zip', path_to_data_to_be_zipped)


In [9]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3)
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

# got this from here:
# https://stackoverflow.com/questions/765736/using-pil-to-make-all-white-pixels-transparent/4531395
def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

In [10]:
def main():
    if not os.path.exists(PATH_TO_STORED_CARTOON_IMAGES):
        print("Can not smooth images, path does not exist: ", PATH_TO_STORED_CARTOON_IMAGES)

    if not os.path.exists(PATH_TO_STORE_SMOOTHED_IMAGES):
        os.makedirs(PATH_TO_STORE_SMOOTHED_IMAGES)

    for filename in os.listdir(PATH_TO_STORED_CARTOON_IMAGES):
        cartoon_images_filename = PATH_TO_STORED_CARTOON_IMAGES + filename
        smoothed_images_filename = PATH_TO_STORE_SMOOTHED_IMAGES + filename

        if not os.path.exists(smoothed_images_filename):
            edge_smoothing(cartoon_images_filename, smoothed_images_filename)
        else:
            print("Skipping file, already exists, ", cartoon_images_filename)

    zip_images(SMOOTHED_IMAGES_ZIPFILE_NAME, PATH_TO_STORE_SMOOTHED_IMAGES)

In [11]:
if __name__ == '__main__':
    main()

Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_a27a46db64cced76057761f83ea4a558ae8263c3.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_5f6be8ca4574d3d92da3a72e7a48dcc5aeb86880.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_75d1ffca17628f730209f20545f59317778f3db1.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_7ef3b0b4ec633f3a1131dcf0490fdb45a428ef3d.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_a4cfc2c77f3c45a3eb3518bc8250ae368d10e7a5.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_27ad65be8a904ae9bd709055345ab5fe3832c117.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_d18249777b813b7957644943f57efe5cf1f89d84.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_ce6e5dd411133ac96cf8bc43f907bac4e9019c5b.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sample_c4cdb5eed27508336a0a523beba5ebd5a2e2d479.jpg
Edge-smoothing of  ./drive/MyDrive/Project/safebooru/sa